In [1]:
%config Completer.use_jedi = False
import pandas as pd
import numpy as np
pd.options.display.max_rows=1000
pd.options.display.max_columns=1000
import math
import re
import datetime as dt
from bureau_fc import get_stats,get_bureau_feats_2
from multiprocessing import Pool
import warnings
warnings.filterwarnings("ignore")
import copy
from functools import partial

In [2]:
DATA_DIR = '/home/sahil/data/av/AV_LTFS3/data/'

In [3]:
train = pd.read_csv(DATA_DIR+'train_Data.csv')
test = pd.read_csv(DATA_DIR+'test_Data.csv')
bur_df = pd.read_pickle(DATA_DIR+'bureau_data.pkl')

In [4]:
train.shape,test.shape,bur_df.shape

((128655, 26), (14745, 25), (591000, 32))

In [5]:
train['DisbursalDate'] = pd.to_datetime(train['DisbursalDate'])
test['DisbursalDate'] = pd.to_datetime(test['DisbursalDate'])
train['MaturityDAte'] = pd.to_datetime(train['MaturityDAte'])
test['MaturityDAte'] = pd.to_datetime(test['MaturityDAte'])


bur_df['DATE-REPORTED'] = pd.to_datetime(bur_df['DATE-REPORTED'])
bur_df['dpd_strin_var'] = bur_df['dpd_string'].fillna('').apply(lambda x: [x[y-3:y] for y in range(3, len(x)+3, 3)] if x.find('E')==-1 else ['000'])

In [6]:
df = pd.concat((train,test),axis=0)
df.shape

(143400, 26)

In [7]:
bur_df = bur_df.merge(df[['ID','DisbursalDate','MaturityDAte']],on='ID')
bur_df.sort_values(['ID','DISBURSED-DT'],inplace=True)

In [8]:
bur_df['dpd_seq'] = bur_df.dpd_strin_var.apply(lambda x: ' '.join(x))

#### using the following combination to treat it as a product
#### can apply the same thing on other sequence as well

In [9]:
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer

In [10]:
def get_tfidf_svd_feats(tfidf_fun,data,apply_svd=True):
    temp = tfidf_fun.fit_transform(data)
#     temp = pd.DataFrame(temp.toarray())
#     temp.columns = ['tfidf_feat_{}'.format(x) for x in range(temp.shape[1])]
    if apply_svd:
        svd = TruncatedSVD(n_components=100,random_state=100)
        temp = pd.DataFrame(svd.fit_transform(temp))
        temp.columns = ['tfidf_svd_feat_{}'.format(x) for x in range(temp.shape[1])]      
        return temp,svd
    else:
        return temp

In [11]:
bur_df['products'] = bur_df['SELF-INDICATOR'].astype('str')+'_'+bur_df['MATCH-TYPE'].astype('str')+'_'+\
bur_df['ACCT-TYPE'].astype('str')+'_'+bur_df['CONTRIBUTOR-TYPE'].astype('str')+'_'+bur_df['OWNERSHIP-IND'].astype('str')
seq_df = bur_df.groupby('ID')['products'].apply(list).reset_index(name='seq')
seq_df.seq = seq_df.seq.apply(lambda x:' '.join(x))
seq_df.seq = seq_df.seq.str.lower()

In [12]:
%%time
vectorizer = TfidfVectorizer(input='content', encoding='utf-8', decode_error='strict', strip_accents=None, 
         lowercase=False, preprocessor=None, tokenizer=None, analyzer='word', stop_words=None, 
         token_pattern='\S+', ngram_range=(1, 3), max_df=0.99, min_df=10, max_features=None, 
         vocabulary=None, binary=False, norm='l2', use_idf=True, 
         smooth_idf=True, sublinear_tf=False)
data,svd = get_tfidf_svd_feats(vectorizer,seq_df.seq)

CPU times: user 19.3 s, sys: 1.05 s, total: 20.4 s
Wall time: 6.57 s


In [13]:
data['ID'] = seq_df['ID']

In [14]:
data.head()

,tfidf_svd_feat_0,tfidf_svd_feat_1,tfidf_svd_feat_2,tfidf_svd_feat_3,tfidf_svd_feat_4,tfidf_svd_feat_5,tfidf_svd_feat_6,tfidf_svd_feat_7,tfidf_svd_feat_8,tfidf_svd_feat_9,tfidf_svd_feat_10,tfidf_svd_feat_11,tfidf_svd_feat_12,tfidf_svd_feat_13,tfidf_svd_feat_14,tfidf_svd_feat_15,tfidf_svd_feat_16,tfidf_svd_feat_17,tfidf_svd_feat_18,tfidf_svd_feat_19,tfidf_svd_feat_20,tfidf_svd_feat_21,tfidf_svd_feat_22,tfidf_svd_feat_23,tfidf_svd_feat_24,tfidf_svd_feat_25,tfidf_svd_feat_26,tfidf_svd_feat_27,tfidf_svd_feat_28,tfidf_svd_feat_29,tfidf_svd_feat_30,tfidf_svd_feat_31,tfidf_svd_feat_32,tfidf_svd_feat_33,tfidf_svd_feat_34,tfidf_svd_feat_35,tfidf_svd_feat_36,tfidf_svd_feat_37,tfidf_svd_feat_38,tfidf_svd_feat_39,tfidf_svd_feat_40,tfidf_svd_feat_41,tfidf_svd_feat_42,tfidf_svd_feat_43,tfidf_svd_feat_44,tfidf_svd_feat_45,tfidf_svd_feat_46,tfidf_svd_feat_47,tfidf_svd_feat_48,tfidf_svd_feat_49,tfidf_svd_feat_50,tfidf_svd_feat_51,tfidf_svd_feat_52,tfidf_svd_feat_53,tfidf_svd_feat_54,tfidf_svd_feat_55,tfidf_svd_feat_56,tfidf_svd_feat_57,tfidf_svd_feat_58,tfidf_svd_feat_59,tfidf_svd_feat_60,tfidf_svd_feat_61,tfidf_svd_feat_62,tfidf_svd_feat_63,tfidf_svd_feat_64,tfidf_svd_feat_65,tfidf_svd_feat_66,tfidf_svd_feat_67,tfidf_svd_feat_68,tfidf_svd_feat_69,tfidf_svd_feat_70,tfidf_svd_feat_71,tfidf_svd_feat_72,tfidf_svd_feat_73,tfidf_svd_feat_74,tfidf_svd_feat_75,tfidf_svd_feat_76,tfidf_svd_feat_77,tfidf_svd_feat_78,tfidf_svd_feat_79,tfidf_svd_feat_80,tfidf_svd_feat_81,tfidf_svd_feat_82,tfidf_svd_feat_83,tfidf_svd_feat_84,tfidf_svd_feat_85,tfidf_svd_feat_86,tfidf_svd_feat_87,tfidf_svd_feat_88,tfidf_svd_feat_89,tfidf_svd_feat_90,tfidf_svd_feat_91,tfidf_svd_feat_92,tfidf_svd_feat_93,tfidf_svd_feat_94,tfidf_svd_feat_95,tfidf_svd_feat_96,tfidf_svd_feat_97,tfidf_svd_feat_98,tfidf_svd_feat_99,ID
0,0.060962,0.061691,0.007819,-0.005386,0.010163,0.122989,-0.031966,0.169891,-0.052629,-0.041681,0.000087,0.000114,0.003125,0.018338,-0.007044,0.002105,-0.000638,-0.011309,-0.000054,0.010194,-0.024599,0.005771,0.060013,0.082291,0.014366,-0.002614,-0.000838,-0.007339,0.001453,-0.024649,0.013288,0.015152,0.009630,-0.000824,0.006813,0.002714,-0.001167,-0.048817,0.044592,-0.023015,0.014851,0.017620,0.045387,-0.030906,0.007173,0.003776,0.015211,-0.020953,0.001952,0.002525,-0.012142,-0.039369,-0.036521,0.006348,-0.004644,0.035691,-0.024049,-0.003855,-0.033096,-0.037986,-0.013133,0.001466,0.024612,0.029046,0.052140,-0.109363,0.061481,0.204481,0.011506,0.108139,-0.091808,0.022072,-0.016145,0.002995,-0.015557,-0.028934,0.015169,-0.062787,-0.069708,0.039640,-0.042153,0.005731,0.013151,-0.000734,0.010738,-0.007060,0.011914,-0.017460,-0.007601,-0.006578,-0.011490,0.038085,0.003787,-0.012301,-0.004903,-0.011139,-0.008844,0.004524,-0.008519,0.011668,1
1,0.061386,0.106169,0.039712,-0.022115,-0.001442,0.256220,-0.133783,0.057254,0.070110,0.368548,-0.032032,-0.029020,-0.009688,-0.002736,-0.010054,0.009549,-0.010733,-0.004520,-0.004680,0.056203,-0.036254,-0.031328,-0.058391,-0.076084,0.004778,0.000173,-0.008377,0.005955,-0.005303,-0.045950,0.040073,0.075139,-0.018612,-0.003450,-0.010910,0.069516,-0.014071,-0.082870,-0.023071,0.010588,0.019175,0.093382,0.085442,0.164878,0.047891,-0.010951,0.004498,0.028886,-0.005660,-0.012144,-0.003479,0.044457,-0.036380,-0.010161,-0.001523,0.089393,-0.010338,0.078346,0.110565,-0.116000,0.080699,0.157722,-0.040778,0.007456,-0.012392,0.074226,-0.026915,0.011818,-0.026203,-0.004931,0.016195,-0.004656,-0.007018,0.022842,0.029457,-0.001146,0.012281,-0.018950,-0.012118,-0.006790,0.009417,-0.013138,0.027137,-0.026554,-0.002295,-0.060466,-0.011760,0.007211,0.002758,-0.026364,-0.036195,-0.002549,-0.120221,-0.004409,0.039388,0.012507,0.021483,0.008412,-0.028538,0.027954,2
2,0.085394,0.080778,0.187741,0.096003,-0.056783,0.185343,-0.092121,0.017749,0.354819,0.051304,0.546795,0.106911,-0.065122,0.042808,-0.002319,-0.034398,0.006515,0.007039,-0.005046,0.126782,0.083252,-0.015792,0.070587,-0.038367,-0.000258,0.038814,-0.001622,-0.045462,-0.046426,-0.000749,-0.007771,-

In [15]:
# %%time
# vectorizer = TfidfVectorizer(input='content', encoding='utf-8', decode_error='strict', strip_accents=None, 
#          lowercase=False, preprocessor=None, tokenizer=None, analyzer='word', stop_words=None, 
#          token_pattern='\S+', ngram_range=(1, 3), max_df=0.99, min_df=100, max_features=None, 
#          vocabulary=None, binary=False, norm='l2', use_idf=True, 
#          smooth_idf=True, sublinear_tf=False)
# data3,svd = get_tfidf_svd_feats(vectorizer,bur_df['dpd_seq'])
# data3['ID'] = bur_df['ID']

In [16]:
# svd.explained_variance_ratio_

In [17]:
# svd.explained_variance_ratio_.sum()

In [18]:
# data3.head()

In [19]:
temp = copy.deepcopy(bur_df)
cond1 = (temp['DISBURSED-DT']>=(temp.DisbursalDate))
temp = temp[cond1]
print(temp.shape)
temp['products'] = temp['SELF-INDICATOR'].astype('str')+'_'+temp['MATCH-TYPE'].astype('str')+'_'+\
temp['ACCT-TYPE'].astype('str')+'_'+temp['CONTRIBUTOR-TYPE'].astype('str')+'_'+temp['OWNERSHIP-IND'].astype('str')
seq_df = temp.groupby('ID')['products'].apply(list).reset_index(name='seq')
seq_df.seq = seq_df.seq.apply(lambda x:' '.join(x))
seq_df.seq = seq_df.seq.str.lower()

(415318, 37)


In [20]:
%%time
vectorizer = TfidfVectorizer(input='content', encoding='utf-8', decode_error='strict', strip_accents=None, 
         lowercase=False, preprocessor=None, tokenizer=None, analyzer='word', stop_words=None, 
         token_pattern='\S+', ngram_range=(1, 3), max_df=0.99, min_df=10, max_features=None, 
         vocabulary=None, binary=False, norm='l2', use_idf=True, 
         smooth_idf=True, sublinear_tf=False)
data2,svd = get_tfidf_svd_feats(vectorizer,seq_df.seq)
data2['ID'] = seq_df['ID']

CPU times: user 17 s, sys: 910 ms, total: 18 s
Wall time: 5.36 s


In [21]:
# %%time
# vectorizer = TfidfVectorizer(input='content', encoding='utf-8', decode_error='strict', strip_accents=None, 
#          lowercase=False, preprocessor=None, tokenizer=None, analyzer='word', stop_words=None, 
#          token_pattern='\S+', ngram_range=(1, 3), max_df=0.99, min_df=100, max_features=None, 
#          vocabulary=None, binary=False, norm='l2', use_idf=True, 
#          smooth_idf=True, sublinear_tf=False)
# data4,svd = get_tfidf_svd_feats(vectorizer,temp['dpd_seq'])
# data4['ID'] = temp['ID']

In [22]:
data2.shape,data.shape
# ,data3.shape,data4.shape

((139994, 101), (143400, 101))

In [23]:
# def get_avg_tfidf(x):
#     _id,m = x
#     _dict = dict(m.iloc[:,:-1].mean())
#     _dict['ID'] = _id
#     return _dict

In [24]:
# %%time
# temp = data4.groupby('ID')
# try:
#     pool = Pool(8) 
#     data_outputs = pool.map(get_avg_tfidf, temp)
# finally: # To make sure processes are closed in the end, even if errors happen
#     pool.close()
#     pool.join()
# data4 = pd.DataFrame(data_outputs)

In [25]:
# data4.shape

In [26]:
# data4.head()

In [27]:
# %%time
# temp = data3.groupby('ID')
# try:
#     pool = Pool(8) 
#     data_outputs = pool.map(get_avg_tfidf, temp)
# finally: # To make sure processes are closed in the end, even if errors happen
#     pool.close()
#     pool.join()
# data3 = pd.DataFrame(data_outputs)

In [28]:
# data3.shape

In [29]:
# data3.head()

In [30]:
data2.columns = ['post_disb_tfidf_'+x if x!='ID' else x for x in data2.columns]
# data3.columns = ['dpd_str_tfidf_'+x if x!='ID' else x for x in data3.columns]
# data4.columns = ['post_disb_dpd_str_tfidf_'+x if x!='ID' else x for x in data4.columns]

In [31]:
# data = data.merge(data2,on='ID').merge(data3,on='ID').merge(data4,on='ID')
data = data.merge(data2,on='ID')

In [32]:
data.head()

,tfidf_svd_feat_0,tfidf_svd_feat_1,tfidf_svd_feat_2,tfidf_svd_feat_3,tfidf_svd_feat_4,tfidf_svd_feat_5,tfidf_svd_feat_6,tfidf_svd_feat_7,tfidf_svd_feat_8,tfidf_svd_feat_9,tfidf_svd_feat_10,tfidf_svd_feat_11,tfidf_svd_feat_12,tfidf_svd_feat_13,tfidf_svd_feat_14,tfidf_svd_feat_15,tfidf_svd_feat_16,tfidf_svd_feat_17,tfidf_svd_feat_18,tfidf_svd_feat_19,tfidf_svd_feat_20,tfidf_svd_feat_21,tfidf_svd_feat_22,tfidf_svd_feat_23,tfidf_svd_feat_24,tfidf_svd_feat_25,tfidf_svd_feat_26,tfidf_svd_feat_27,tfidf_svd_feat_28,tfidf_svd_feat_29,tfidf_svd_feat_30,tfidf_svd_feat_31,tfidf_svd_feat_32,tfidf_svd_feat_33,tfidf_svd_feat_34,tfidf_svd_feat_35,tfidf_svd_feat_36,tfidf_svd_feat_37,tfidf_svd_feat_38,tfidf_svd_feat_39,tfidf_svd_feat_40,tfidf_svd_feat_41,tfidf_svd_feat_42,tfidf_svd_feat_43,tfidf_svd_feat_44,tfidf_svd_feat_45,tfidf_svd_feat_46,tfidf_svd_feat_47,tfidf_svd_feat_48,tfidf_svd_feat_49,tfidf_svd_feat_50,tfidf_svd_feat_51,tfidf_svd_feat_52,tfidf_svd_feat_53,tfidf_svd_feat_54,tfidf_svd_feat_55,tfidf_svd_feat_56,tfidf_svd_feat_57,tfidf_svd_feat_58,tfidf_svd_feat_59,tfidf_svd_feat_60,tfidf_svd_feat_61,tfidf_svd_feat_62,tfidf_svd_feat_63,tfidf_svd_feat_64,tfidf_svd_feat_65,tfidf_svd_feat_66,tfidf_svd_feat_67,tfidf_svd_feat_68,tfidf_svd_feat_69,tfidf_svd_feat_70,tfidf_svd_feat_71,tfidf_svd_feat_72,tfidf_svd_feat_73,tfidf_svd_feat_74,tfidf_svd_feat_75,tfidf_svd_feat_76,tfidf_svd_feat_77,tfidf_svd_feat_78,tfidf_svd_feat_79,tfidf_svd_feat_80,tfidf_svd_feat_81,tfidf_svd_feat_82,tfidf_svd_feat_83,tfidf_svd_feat_84,tfidf_svd_feat_85,tfidf_svd_feat_86,tfidf_svd_feat_87,tfidf_svd_feat_88,tfidf_svd_feat_89,tfidf_svd_feat_90,tfidf_svd_feat_91,tfidf_svd_feat_92,tfidf_svd_feat_93,tfidf_svd_feat_94,tfidf_svd_feat_95,tfidf_svd_feat_96,tfidf_svd_feat_97,tfidf_svd_feat_98,tfidf_svd_feat_99,ID,post_disb_tfidf_tfidf_svd_feat_0,post_disb_tfidf_tfidf_svd_feat_1,post_disb_tfidf_tfidf_svd_feat_2,post_disb_tfidf_tfidf_svd_feat_3,post_disb_tfidf_tfidf_svd_feat_4,post_disb_tfidf_tfidf_svd_feat_5,post_disb_tfidf_tfidf_svd_feat_6,post_disb_tfidf_tfidf_svd_feat_7,post_disb_tfidf_tfidf_svd_feat_8,post_disb_tfidf_tfidf_svd_feat_9,post_disb_tfidf_tfidf_svd_feat_10,post_disb_tfidf_tfidf_svd_feat_11,post_disb_tfidf_tfidf_svd_feat_12,post_disb_tfidf_tfidf_svd_feat_13,post_disb_tfidf_tfidf_svd_feat_14,post_disb_tfidf_tfidf_svd_feat_15,post_disb_tfidf_tfidf_svd_feat_16,post_disb_tfidf_tfidf_svd_feat_17,post_disb_tfidf_tfidf_svd_feat_18,post_disb_tfidf_tfidf_svd_feat_19,post_disb_tfidf_tfidf_svd_feat_20,post_disb_tfidf_tfidf_svd_feat_21,post_disb_tfidf_tfidf_svd_feat_22,post_disb_tfidf_tfidf_svd_feat_23,post_disb_tfidf_tfidf_svd_feat_24,post_disb_tfidf_tfidf_svd_feat_25,post_disb_tfidf_tfidf_svd_feat_26,post_disb_tfidf_tfidf_svd_feat_27,post_disb_tfidf_tfidf_svd_feat_28,post_disb_tfidf_tfidf_svd_feat_29,post_disb_tfidf_tfidf_svd_feat_30,post_disb_tfidf_tfidf_svd_feat_31,post_disb_tfidf_tfidf_svd_feat_32,post_disb_tfidf_tfidf_svd_feat_33,post_disb_tfidf_tfidf_svd_feat_34,post_disb_tfidf_tfidf_svd_feat_35,post_disb_tfidf_tfidf_svd_feat_36,post_disb_tfidf_tfidf_svd_feat_37,post_disb_tfidf_tfidf_svd_feat_38,post_disb_tfidf_tfidf_svd_feat_39,post_disb_tfidf_tfidf_svd_feat_40,post_disb_tfidf_tfidf_svd_feat_41,post_disb_tfidf_tfidf_svd_feat_42,post_disb_tfidf_tfidf_svd_feat_43,post_disb_tfidf_tfidf_svd_feat_44,post_disb_tfidf_tfidf_svd_feat_45,post_disb_tfidf_tfidf_svd_feat_46,post_disb_tfidf_tfidf_svd_feat_47,post_disb_tfidf_tfidf_svd_feat_48,post_disb_tfidf_tfidf_svd_feat_49,post_disb_tfidf_tfidf_svd_feat_50,post_disb_tfidf_tfidf_svd_feat_51,post_disb_tfidf_tfidf_svd_feat_52,post_disb_tfidf_tfidf_svd_feat_53,post_disb_tfidf_tfidf_svd_feat_54,post_disb_tfidf_tfidf_svd_feat_55,post_disb_tfidf_tfidf_svd_feat_56,post_disb_tfidf_tfidf_svd_feat_57,post_disb_tfidf_tfidf_svd_feat_58,post_disb_tfidf_tfidf_svd_feat_59,post_disb_tfidf_tfidf_svd_feat_60,post_disb_tfidf_tfidf_svd_feat_61,post_disb_tfidf_tfidf_svd_feat_62,post_disb_tfidf_tfidf_svd_feat_63,post_disb_tfidf_tfidf_svd_feat_64,post_d

In [33]:
for col in data.columns:
    if col!='ID':
        data[col] = data[col].astype('float32')

In [34]:
data.head()

,tfidf_svd_feat_0,tfidf_svd_feat_1,tfidf_svd_feat_2,tfidf_svd_feat_3,tfidf_svd_feat_4,tfidf_svd_feat_5,tfidf_svd_feat_6,tfidf_svd_feat_7,tfidf_svd_feat_8,tfidf_svd_feat_9,tfidf_svd_feat_10,tfidf_svd_feat_11,tfidf_svd_feat_12,tfidf_svd_feat_13,tfidf_svd_feat_14,tfidf_svd_feat_15,tfidf_svd_feat_16,tfidf_svd_feat_17,tfidf_svd_feat_18,tfidf_svd_feat_19,tfidf_svd_feat_20,tfidf_svd_feat_21,tfidf_svd_feat_22,tfidf_svd_feat_23,tfidf_svd_feat_24,tfidf_svd_feat_25,tfidf_svd_feat_26,tfidf_svd_feat_27,tfidf_svd_feat_28,tfidf_svd_feat_29,tfidf_svd_feat_30,tfidf_svd_feat_31,tfidf_svd_feat_32,tfidf_svd_feat_33,tfidf_svd_feat_34,tfidf_svd_feat_35,tfidf_svd_feat_36,tfidf_svd_feat_37,tfidf_svd_feat_38,tfidf_svd_feat_39,tfidf_svd_feat_40,tfidf_svd_feat_41,tfidf_svd_feat_42,tfidf_svd_feat_43,tfidf_svd_feat_44,tfidf_svd_feat_45,tfidf_svd_feat_46,tfidf_svd_feat_47,tfidf_svd_feat_48,tfidf_svd_feat_49,tfidf_svd_feat_50,tfidf_svd_feat_51,tfidf_svd_feat_52,tfidf_svd_feat_53,tfidf_svd_feat_54,tfidf_svd_feat_55,tfidf_svd_feat_56,tfidf_svd_feat_57,tfidf_svd_feat_58,tfidf_svd_feat_59,tfidf_svd_feat_60,tfidf_svd_feat_61,tfidf_svd_feat_62,tfidf_svd_feat_63,tfidf_svd_feat_64,tfidf_svd_feat_65,tfidf_svd_feat_66,tfidf_svd_feat_67,tfidf_svd_feat_68,tfidf_svd_feat_69,tfidf_svd_feat_70,tfidf_svd_feat_71,tfidf_svd_feat_72,tfidf_svd_feat_73,tfidf_svd_feat_74,tfidf_svd_feat_75,tfidf_svd_feat_76,tfidf_svd_feat_77,tfidf_svd_feat_78,tfidf_svd_feat_79,tfidf_svd_feat_80,tfidf_svd_feat_81,tfidf_svd_feat_82,tfidf_svd_feat_83,tfidf_svd_feat_84,tfidf_svd_feat_85,tfidf_svd_feat_86,tfidf_svd_feat_87,tfidf_svd_feat_88,tfidf_svd_feat_89,tfidf_svd_feat_90,tfidf_svd_feat_91,tfidf_svd_feat_92,tfidf_svd_feat_93,tfidf_svd_feat_94,tfidf_svd_feat_95,tfidf_svd_feat_96,tfidf_svd_feat_97,tfidf_svd_feat_98,tfidf_svd_feat_99,ID,post_disb_tfidf_tfidf_svd_feat_0,post_disb_tfidf_tfidf_svd_feat_1,post_disb_tfidf_tfidf_svd_feat_2,post_disb_tfidf_tfidf_svd_feat_3,post_disb_tfidf_tfidf_svd_feat_4,post_disb_tfidf_tfidf_svd_feat_5,post_disb_tfidf_tfidf_svd_feat_6,post_disb_tfidf_tfidf_svd_feat_7,post_disb_tfidf_tfidf_svd_feat_8,post_disb_tfidf_tfidf_svd_feat_9,post_disb_tfidf_tfidf_svd_feat_10,post_disb_tfidf_tfidf_svd_feat_11,post_disb_tfidf_tfidf_svd_feat_12,post_disb_tfidf_tfidf_svd_feat_13,post_disb_tfidf_tfidf_svd_feat_14,post_disb_tfidf_tfidf_svd_feat_15,post_disb_tfidf_tfidf_svd_feat_16,post_disb_tfidf_tfidf_svd_feat_17,post_disb_tfidf_tfidf_svd_feat_18,post_disb_tfidf_tfidf_svd_feat_19,post_disb_tfidf_tfidf_svd_feat_20,post_disb_tfidf_tfidf_svd_feat_21,post_disb_tfidf_tfidf_svd_feat_22,post_disb_tfidf_tfidf_svd_feat_23,post_disb_tfidf_tfidf_svd_feat_24,post_disb_tfidf_tfidf_svd_feat_25,post_disb_tfidf_tfidf_svd_feat_26,post_disb_tfidf_tfidf_svd_feat_27,post_disb_tfidf_tfidf_svd_feat_28,post_disb_tfidf_tfidf_svd_feat_29,post_disb_tfidf_tfidf_svd_feat_30,post_disb_tfidf_tfidf_svd_feat_31,post_disb_tfidf_tfidf_svd_feat_32,post_disb_tfidf_tfidf_svd_feat_33,post_disb_tfidf_tfidf_svd_feat_34,post_disb_tfidf_tfidf_svd_feat_35,post_disb_tfidf_tfidf_svd_feat_36,post_disb_tfidf_tfidf_svd_feat_37,post_disb_tfidf_tfidf_svd_feat_38,post_disb_tfidf_tfidf_svd_feat_39,post_disb_tfidf_tfidf_svd_feat_40,post_disb_tfidf_tfidf_svd_feat_41,post_disb_tfidf_tfidf_svd_feat_42,post_disb_tfidf_tfidf_svd_feat_43,post_disb_tfidf_tfidf_svd_feat_44,post_disb_tfidf_tfidf_svd_feat_45,post_disb_tfidf_tfidf_svd_feat_46,post_disb_tfidf_tfidf_svd_feat_47,post_disb_tfidf_tfidf_svd_feat_48,post_disb_tfidf_tfidf_svd_feat_49,post_disb_tfidf_tfidf_svd_feat_50,post_disb_tfidf_tfidf_svd_feat_51,post_disb_tfidf_tfidf_svd_feat_52,post_disb_tfidf_tfidf_svd_feat_53,post_disb_tfidf_tfidf_svd_feat_54,post_disb_tfidf_tfidf_svd_feat_55,post_disb_tfidf_tfidf_svd_feat_56,post_disb_tfidf_tfidf_svd_feat_57,post_disb_tfidf_tfidf_svd_feat_58,post_disb_tfidf_tfidf_svd_feat_59,post_disb_tfidf_tfidf_svd_feat_60,post_disb_tfidf_tfidf_svd_feat_61,post_disb_tfidf_tfidf_svd_feat_62,post_disb_tfidf_tfidf_svd_feat_63,post_disb_tfidf_tfidf_svd_feat_64,post_d

In [35]:
data.to_pickle('tfidf_feats1.pkl')